# Libraries

In [1]:
from  google.colab  import  drive 
drive.mount ('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import scipy
import math
 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,balanced_accuracy_score
from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics.pairwise as pw
from sklearn.metrics.pairwise import pairwise_distances
from mlxtend.evaluate import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import confusion_matrix


import matplotlib.pyplot as plt  
%matplotlib inline    
import matplotlib as mpl
import seaborn as sns

import random
from scipy import sparse
from scipy.stats import pearsonr
import re

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image

import nltk
from nltk.corpus import stopwords

import torch
from torchtext import data
from torchtext import datasets

# Recommend with SVD and Emotions

## Emotion Analysis

Εξάγουμε το συναίσθημα από το input του χρήστη, και βρίσκουμε τα ξενοδοχεία που προσφέρουν αυτό το συναίσθημα περισσότερο.

In [3]:
df_emotions = pd.read_csv('/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Accommodation Reviews/reviews_with_emotions_full.csv')
df_emotions.drop(["Unnamed: 0"], axis = 1, inplace=True)

In [4]:
username = input ("Username: ")
password = input ("Password: ")

Username: 1000
Password: 2


In [5]:
users_comments = df_emotions[df_emotions['UserId'] == int(username)]

In [6]:
user_positive_comments = users_comments[users_comments['Rating'] > 3]

In [7]:
already_visited = users_comments["Name"].values

In [8]:
user_positive_comments = user_positive_comments.groupby("Emotion").count()
user_positive_comments = user_positive_comments.sort_values(by=['Emotion'], ascending = True)

In [9]:
most_felt_emotion = user_positive_comments.index.values[0]

In [10]:
possible_options = df_emotions[df_emotions['Emotion'] == most_felt_emotion]

In [11]:
recommends_df = possible_options.groupby("Name")['Emotion'].count().to_frame()
recommends_df = recommends_df.sort_values(by=['Emotion'], ascending = False)

In [12]:
possible_recommendations = recommends_df.index.values

In [13]:
recommendations_from_emotion = []
for i in possible_recommendations:
  if i not in already_visited:
    recommendations_from_emotion.append(i)

## SVD Implementation

In [22]:
# Δεν έχει σημασία πιο dataset θα χρησιμοποιήσουμε.
# Η μόνη προϋπόθεση είναι να υπάρχει το όνομα του ξενοδοχείου, το UserId και το rating για να γίνει το pivot.
df = pd.read_csv('/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Accommodation Reviews/hotels_for_keywords_recommendation.csv')

In [23]:
pivot = pd.pivot_table(df, index='UserId', columns=['Name'], values='Rating').fillna(0)
pivot.head(5)

Name,InterContinental,amalia,athens_gate,attalos,ava,central,cypria,electra_metropolis,grand_hyatt,hera,...,innathens,marble,melia,new,plaka_hotel,polis grand,president,royal,titania,wyndham
UserId,,,,,,,,,,,,,,,,,,,,,
1,5.0,5.0,5.0,5.0,5.0,4.0,4.0,5.0,5.0,5.0,...,5.0,5.0,4.0,5.0,5.0,5.0,3.0,3.0,2.0,5.0
2,5.0,4.0,5.0,5.0,5.0,4.0,3.0,5.0,3.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,3.0,1.0,5.0
3,5.0,5.0,5.0,5.0,5.0,1.0,4.0,4.0,4.0,2.0,...,5.0,5.0,5.0,5.0,5.0,5.0,2.0,4.0,5.0,5.0
4,4.0,5.0,5.0,5.0,5.0,4.0,5.0,5.0,3.0,3.0,...,5.0,5.0,3.0,5.0,5.0,5.0,1.0,5.0,5.0,5.0
5,5.0,5.0,5.0,4.0,5.0,5.0,4.0,5.0,1.0,4.0,...,4.0,5.0,5.0,5.0,5.0,5.0,3.0,2.0,3.0,2.0


In [24]:
pivot_mat = pivot.values
pivot_mat[:5]

array([[5., 5., 5., 5., 5., 4., 4., 5., 5., 5., 5., 5., 5., 4., 5., 5.,
        5., 3., 3., 2., 5.],
       [5., 4., 5., 5., 5., 4., 3., 5., 3., 5., 5., 5., 5., 5., 5., 5.,
        5., 5., 3., 1., 5.],
       [5., 5., 5., 5., 5., 1., 4., 4., 4., 2., 5., 5., 5., 5., 5., 5.,
        5., 2., 4., 5., 5.],
       [4., 5., 5., 5., 5., 4., 5., 5., 3., 3., 5., 5., 5., 3., 5., 5.,
        5., 1., 5., 5., 5.],
       [5., 5., 5., 4., 5., 5., 4., 5., 1., 4., 4., 4., 5., 5., 5., 5.,
        5., 3., 2., 3., 2.]])

In [25]:
user_id = list(pivot.index)
user_id[:10]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [26]:
sparse_matrix = csr_matrix(pivot_mat)
sparse_matrix

<2000x21 sparse matrix of type '<class 'numpy.float64'>'
	with 23771 stored elements in Compressed Sparse Row format>

In [27]:
factor_n = 15
U, sigma, V = svds(sparse_matrix, k = factor_n)

In [28]:
print(U.shape)
print(V.shape)
sigma = np.diag(sigma)
print(sigma.shape)

(2000, 15)
(15, 21)
(15, 15)


In [29]:
pred_rating = np.dot(np.dot(U,sigma),V)
pred_rating[:5]

array([[4.99832997, 4.92584418, 4.36498047, 5.10660679, 4.8304552 ,
        4.0952812 , 4.57435412, 5.07475355, 4.80435492, 4.96769162,
        5.02832549, 5.40037571, 4.74818008, 4.10754589, 4.84428258,
        4.9057474 , 4.47455108, 3.13580623, 2.97786085, 2.9517599 ,
        4.99430066],
       [5.01451701, 3.76037842, 4.11797798, 5.18307432, 4.62448293,
        4.31094733, 3.85917719, 5.06620363, 3.98342102, 4.89381825,
        5.04281067, 4.54142196, 4.29707693, 5.21657211, 5.00330277,
        4.8759612 , 4.40205337, 5.06252302, 2.97481653, 2.71332585,
        4.77953511],
       [4.50198656, 5.06006484, 5.65763734, 5.07869698, 5.31187683,
        0.72211176, 4.36087175, 4.69857604, 4.18722196, 3.15441212,
        5.03637074, 4.78245944, 4.65807836, 4.83249516, 3.86145133,
        4.58010945, 4.93500406, 2.35808003, 3.93999325, 4.56671589,
        4.64494022],
       [3.65683058, 5.02918369, 5.72357695, 5.02994843, 5.22380642,
        3.82638914, 5.06784072, 5.17581742, 3.4852447

In [30]:
pred_rating_n = (pred_rating - pred_rating.min()) / (pred_rating.max() - pred_rating.min())

In [31]:
pred_rating_n[:5]

array([[0.85942283, 0.84941273, 0.77195886, 0.87437559, 0.83623975,
        0.73471407, 0.80087284, 0.86997674, 0.83263537, 0.85519175,
        0.86356514, 0.91494433, 0.82487776, 0.73640779, 0.83814927,
        0.84663741, 0.78709029, 0.602213  , 0.58040114, 0.57679666,
        0.85886639],
       [0.86165822, 0.68846484, 0.73784844, 0.88493556, 0.8077955 ,
        0.76449702, 0.7021087 , 0.86879601, 0.71926646, 0.84499003,
        0.8655655 , 0.79632499, 0.76258155, 0.88956152, 0.86010956,
        0.84252401, 0.77707854, 0.86828773, 0.57998073, 0.54386952,
        0.82920781],
       [0.79087906, 0.86794826, 0.95047152, 0.87052131, 0.90272286,
        0.26888788, 0.77139146, 0.81802757, 0.74741086, 0.60478242,
        0.86467617, 0.82961166, 0.81243494, 0.83652146, 0.70242276,
        0.80166763, 0.85067768, 0.49481097, 0.7132692 , 0.79981801,
        0.8106206 ],
       [0.67416515, 0.86368365, 0.95957761, 0.86378926, 0.89056056,
        0.69758076, 0.86902209, 0.8839334 , 0.6504695

In [32]:
pred_df = pd.DataFrame(pred_rating_n, columns = pivot.columns, index=user_id).transpose()
pred_df.head(15)

,1,2,3,4,5,6,7,8,9,10,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
Name,,,,,,,,,,,,,,,,,,,,,
InterContinental,0.859423,0.861658,0.790879,0.674165,0.841006,0.866478,0.822680,0.887463,0.349938,0.738375,...,0.167509,0.167095,0.168412,0.167095,0.168827,0.168412,0.167509,0.167095,0.169730,0.166191
amalia,0.849413,0.688465,0.867948,0.863684,0.839419,0.696146,0.581101,0.412209,0.811083,0.842750,...,0.169566,0.169666,0.169712,0.169666,0.169612,0.169712,0.169566,0.169666,0.169757,0.169521
athens_gate,0.771959,0.737848,0.950472,0.959578,0.839550,0.722244,0.958861,0.742713,0.694351,0.674218,...,0.170040,0.170258,0.171318,0.170258,0.171100,0.171318,0.170040,0.170258,0.172379,0.168979
attalos,0.874376,0.884936,0.870521,0.863789,0.735459,0.874432,0.861847,0.750510,0.733733,0.733310,...,0.169186,0.169190,0.168830,0.169190,0.168825,0.168830,0.169186,0.169190,0.168470,0.169546
ava,0.836240,0.807795,0.902723,0.890561,0.843002,0.825341,0.878944,0.773778,0.783646,0.825688,...,0.170695,0.171078,0.170487,0.171078,0.170105,0.170487,0.170695,0.171078,0.169896,0.171286
central,0.734714,0.764497,0.268888,0.697581,0.878148,0.893358,0.296321,0.778504,0.800304,0.749201,...,0.167957,0.167655,0.168156,0.167655,0.168458,0.168156,0.167957,0.167655,0.168656,0.167457
cypria,0.800873,0.702109,0.771391,0.869022,0.761802,0.885501,0.710248,0.485947,0.711537,0.859855,...,0.167130,0.166621,0.166580,0.166621,0.167089,0.166580,0.167130,0.166621,0.166540,0.167170
electra_metropolis,0.869977,0.868796,0.818028,0.883933,0.881078,0.866180,0.882638,0.595141,0.851155,0.864892,...,0.170672,0.171049,0.170567,0.171049,0.170190,0.170567,0.170672,0.171049,0.170085,0.171154
grand_hyatt,0.832635,0.719266,0.747411,0.650470,0.448755,0.688753,0.688741,0.738187,0.558833,0.679579,...,0.167792,0.167449,0.167549,0.167449,0.167893,0.167549,0.167792,0.167449,0.167650,0.167692


In [34]:
def recommend_hotels(user_id ,pred_df, items_df, hotels_to_ignore=[], top_list=20, verbose=False):

        sorted_user_predictions = pred_df[user_id].sort_values(ascending=False).reset_index().rename(columns={user_id: 'recStrength'})

        recommendations_df = sorted_user_predictions[~sorted_user_predictions['Name'].isin(hotels_to_ignore)].sort_values(by='recStrength', ascending = False).head(top_list)
        print(recommendations_df)
        return recommendations_df


In [40]:
def recommender(user_id,pred_df,real_df,df_meta):

    restaurants_to_ignore = df[df['UserId'] == str(user_id)]['Name'].values
    recommend = recommend_hotels(user_id, pred_df,real_df, restaurants_to_ignore)
    df_user= real_df.loc[real_df['UserId'] == user_id]
    new_df = df_user.merge(recommend, how = 'outer', left_on = 'Name', right_on = 'Name')
    rec_df = new_df.loc[new_df['Rating'].isnull()==True]
    df_rec_t = rec_df.loc[:, ['Name', 'recStrength']]
    df_last = pd.merge(df_rec_t,df_meta[['HotelId','Name']],on=['Name'], how='left')
    df_last = df_last.drop_duplicates(subset=["Name", "HotelId"], keep='first')
    df_last = df_last.reset_index(drop=True) 
    return df_last

In [41]:
hotels = pd.read_csv('/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Accommodation Reviews/hotels.csv')
ratings = pd.read_csv('/content/drive/MyDrive/ΕΙΔΙΚΟ ΘΕΜΑ/Accommodation Reviews/ratings.csv')
df_meta = hotels.merge(ratings, how="left", on="HotelId")
df_meta = df_meta.astype({"UserId": str, "HotelId": str})
df_meta.head()

,HotelId,Name,UserId,Review,Rating
0,1,amalia,1,"Amazing hotel, great location and helpful staf...",5
1,1,amalia,2,"For a hotel in a world capital, our room was d...",4
2,1,amalia,3,We are here for 6 nights and found this Hotel ...,5
3,1,amalia,4,"Clean facilities, rooms. Great customer servic...",5
4,1,amalia,5,I have stayed In Athens in several hotels and...,5


In [42]:
df = df.astype({"UserId": str})
df

,UserId,Name,Review,Rating,Emotion,sentiment,Review_Clean,Reviewer_Count,Restaurant_Count,word_without_stop,positive
0,1,amalia,"amazing hotel, great location and helpful staf...",5,admiration,2,amazing hotel great location and helpful staff...,21,1378,amazing hotel great location helpful staff fac...,True
1,462,herodion,stayed here for 4 nights. arrived late past mi...,5,approval,2,stayed here for 4 nights arrived late past mid...,21,2000,stayed 4 nights arrived late past midnight giv...,True
2,451,herodion,this place is pretty much perfect for those in...,5,admiration,2,this place is pretty much perfect for those in...,21,2000,place pretty much perfect interested acropolis...,True
3,452,herodion,the hotel itself is average. the rooms are sma...,3,admiration,1,the hotel itself is average the rooms are smal...,21,2000,hotel average rooms small halls felt odd said ...,True
4,453,herodion,the lovely and classic hotel herodion pleased ...,5,admiration,2,the lovely and classic hotel herodion pleased ...,21,2000,lovely classic hotel herodion pleased every wa...,True
...,...,...,...,...,...,...,...,...,...,...,...
23766,1672,royal,my husband and i visited the royal olympic hot...,4,approval,2,my husband and i visited the royal olympic hot...,2,2000,husband visited royal olympic hotel end august...,True
23767,1671,royal,hotel is only 10 mins walking time to the hist...,3,admiration,1,hotel is only 10 mins walking time to the hist...,2,2000,hotel 10 mins walking time historical sites ro...,True
23768,1670,royal,make no mistake about it this hotel is a great...,3,admiration,1,make no mistake about it this hotel is a great...,2,2000,make mistake hotel great base see top attracti...,True
23769,1669,royal,i stayed at royal olympic from sunday and chec...,5,approval,2,i stayed at royal olympic from sunday and chec...,2,2000,stayed royal olympic sunday checked wednesday ...,True


In [43]:
recommendations_from_SVD = recommender(int(username),pred_df,df,df_meta)

           Name  recStrength
13      titania     0.195052
14    innathens     0.190553
15       cypria     0.189853
16      wyndham     0.174804
17       marble     0.167774
18  athens_gate     0.158412
19  grand_hyatt     0.156586
20  polis grand     0.131147


## Combinign SVD and Emotion Analysis Results

Τώρα, συνδυάζουμε τα recommendations από τις 2 μεθόδους.
Αυτό γίνεται αυξάνοντας το strength που προέρχεται από το SVD, κατά ένα ποσοστό αναλόγως το συναίσθημα που προσφέρει το κάθε εστιατόριο.
Για παράδειγμα, για το πρώτο εστιατόριο που προτείνεται με βάση το συναίσθημα, αυξάνουμε το strength κατά 20%. Για το δεύτερο, το αυξάνουμε κατά 10% ενώ το τρίτο κατά 5%.

In [44]:
recommendations_from_emotion

['wyndham',
 'polis grand',
 'athens_gate',
 'cypria',
 'innathens',
 'titania',
 'marble',
 'grand_hyatt']

In [45]:
recommendations_from_SVD

,Name,recStrength,HotelId
0,titania,0.195052,20
1,innathens,0.190553,11
2,cypria,0.189853,6
3,wyndham,0.174804,21
4,marble,0.167774,13
5,athens_gate,0.158412,2
6,grand_hyatt,0.156586,8
7,polis grand,0.131147,17


In [46]:
score_dict = {}

for index, row in recommendations_from_SVD.iterrows():
  name = row['Name']
  print("Name is ", name)
  recStrength = row['recStrength']
  print("recStrength is ", recStrength)
  if(len(recommendations_from_emotion) >= 3):
    if(name == recommendations_from_emotion[0]):
        print('1')
        recStrength = recStrength + 0.2*recStrength
        print("new recStrength is ", recStrength)
    elif(name == recommendations_from_emotion[1]):
        recStrength = recStrength + 0.1*recStrength
        print("new recStrength is ", recStrength)
    elif(name == recommendations_from_emotion[2]):
        recStrength = recStrength + 0.05*recStrength
        print("new recStrength is ", recStrength)
  elif(len(recommendations_from_emotion) == 2):
    if(name == recommendations_from_emotion[0]):  
      recStrength = recStrength + 0.2*recStrength
      print("new recStrength is ", recStrength)
    elif(name == recommendations_from_emotion[1]):
      recStrength = recStrength + 0.1*recStrength
      print("new recStrength is ", recStrength)
  elif(len(recommendations_from_emotion) == 1):
    if(name == recommendations_from_emotion[0]):
      recStrength = recStrength + 0.2*recStrength
      print("new recStrength is ", recStrength)
  print('\n')
  score_dict[index] = recStrength    


Name is  titania
recStrength is  0.19505190818154908


Name is  innathens
recStrength is  0.190552553052133


Name is  cypria
recStrength is  0.1898533194294804


Name is  wyndham
recStrength is  0.17480404861288534
1
new recStrength is  0.2097648583354624


Name is  marble
recStrength is  0.1677742025291645


Name is  athens_gate
recStrength is  0.15841185170961936
new recStrength is  0.16633244429510033


Name is  grand_hyatt
recStrength is  0.15658606159881072


Name is  polis grand
recStrength is  0.13114678848577202
new recStrength is  0.14426146733434922




In [47]:
score_dict

{0: 0.19505190818154908,
 1: 0.190552553052133,
 2: 0.1898533194294804,
 3: 0.2097648583354624,
 4: 0.1677742025291645,
 5: 0.16633244429510033,
 6: 0.15658606159881072,
 7: 0.14426146733434922}

In [48]:
import operator
sorted_scores = sorted(score_dict.items(), key=operator.itemgetter(1), reverse=True)
sorted_scores

[(3, 0.2097648583354624),
 (0, 0.19505190818154908),
 (1, 0.190552553052133),
 (2, 0.1898533194294804),
 (4, 0.1677742025291645),
 (5, 0.16633244429510033),
 (6, 0.15658606159881072),
 (7, 0.14426146733434922)]

In [49]:
counter = 0
#δημιουργία νέου dataframe για την αποθήκευση των recommendations
recommendations = pd.DataFrame(columns=('Name', 'score'))
# τα reviews με τα υψηλότερα cosign similarity
for i in sorted_scores:
  recommendations = recommendations.append({'Name': recommendations_from_SVD.iloc[i[0]]['Name'],
                          'score': i[1]}, ignore_index=True)
            
  counter += 1
  if counter>10:
    break

In [50]:
recommendations

,Name,score
0,wyndham,0.209765
1,titania,0.195052
2,innathens,0.190553
3,cypria,0.189853
4,marble,0.167774
5,athens_gate,0.166332
6,grand_hyatt,0.156586
7,polis grand,0.144261
